## Enviorment

In [2]:
!whoami

michael_zl_prime


In [3]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       312164605303-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [4]:
#which enciorment/virtualenv running in
import sys
sys.prefix

'/usr'

In [5]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all --purge_orphaned_data True > /tmp/tensorboard_run.log

---

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import ipywidgets as widgets
import datetime
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage

from config import *
from models.six_stage_linear_model import ModelMaker
import dataset_functions
import visualizations as v

print(tf.version.VERSION)

2.1.0-dev20191124


In [11]:
steps_per_epoch=int(DATASET_SIZE/BATCH_SIZE)

---
# Make dataset

Examine datasets

---
# Callbacks
**Checkpoints**

In [26]:
#local dir for CPU/GPU training
#checkpoints_dir='./checkpoints'
#!mkdir checkpoints
#gcs dir for TPU training

#make sure the directory exists, if not make it.

'gs://dl_training_results/checkpoints/ModelWeights-01Sun1219-0027-{epoch:04d}.ckpt'

**Logging**

**TensorBoard**

In [34]:
#!gsutil -m rm -r {tesnroboard_base_dir}

In [37]:
#the command to run tensorboard
#tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

In [39]:
callbacks=[
    cp_callback
    #,csv_logger  #should open an issue on that
    ,tensorboard_callback
    ,tf.keras.callbacks.LearningRateScheduler(get_lr)
    ,PrintLR()
    ,tf.keras.callbacks.TerminateOnNaN()
]

---
# Model

## Load weights

In [41]:
#delete checkpoints if needed
#!gsutil -m rm -r {checkpoints_dir}

In [42]:
chekpoints_bucket="dl_training_results"
for i,f in enumerate( storage_client.list_blobs(chekpoints_bucket,prefix='checkpoints')):
    if f.name.endswith('.index'):
        print("gs://"+chekpoints_bucket+"/"+f.name[:-6])

gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-1956-0001.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2025-0001.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2048-0001.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2048-0002.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2048-0003.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2048-0004.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2048-0005.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2231-0004.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0004.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0005.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0006.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0007.ckpt
gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0008.ckpt

In [51]:
cpkt="gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0007.ckpt"
starting_epoch=7

In [44]:
# !mkdir /tmp/ckpt
# !gsutil cp "gs://dl_training_results/checkpoints/ModelWeights-30(Sat)1119-2244-0006.*" /tmp/ckpt/
# cpkt="/tmp/ckpt/ModelWeights-30(Sat)1119-2244-0006.ckpt"

In [45]:
load=widgets.Checkbox(
    value=False,
    description='Load weights'
)
load

Checkbox(value=False, description='Load weights')

In [52]:
model_maker=ModelMaker() #must be outside scope to keep the graph clean
def define():
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics={'stage1paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage2paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage3paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage4paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage5heatmap_output': [
             #AnalogRecall(),
             tf.keras.metrics.MeanAbsoluteError()]    
         ,'stage6heatmap_output': [
             #AnalogRecall(),
             tf.keras.metrics.MeanAbsoluteError()]
        }
    
    train_model.compile(optimizer=tf.keras.optimizers.Adam()
                    ,loss=tf.keras.losses.MeanSquaredError(name="MSE_loss")
                    ,metrics=metrics                    
                   )
    return train_model,test_model

if USING_TPU:
    with strategy.scope():
        train_model,test_model=define()
        if (load.value):
            train_model.load_weights(cpkt)
else:
    train_model,test_model=define()
    if (load.value):
        train_model.load_weights(cpkt)

---
Actually training

In [53]:
train_history=train_model.fit(
    ds
    ,epochs=10
    ,steps_per_epoch=steps_per_epoch
    ,validation_data=ds_v
    ,callbacks=callbacks
    ,initial_epoch=starting_epoch
)

Train for 437 steps

Learning rate for epoch 7 is 0.003000000026077032
Epoch 8/10
281/437 [==================>...........] - ETA: 3:25 - loss: 3513169370137980.5000 - stage1paf_output_loss: 1.5843 - stage2paf_output_loss: 1785.4202 - stage3paf_output_loss: 796023.0625 - stage4paf_output_loss: 377732320.0000 - stage5heatmap_output_loss: 162006532096.0000 - stage6heatmap_output_loss: 3513007028043776.0000 - stage1paf_output_mean_absolute_error: 0.0245 - stage2paf_output_mean_absolute_error: 0.6583 - stage3paf_output_mean_absolute_error: 18.2988 - stage4paf_output_mean_absolute_error: 463.9840 - stage5heatmap_output_mean_absolute_error: 12686.9766 - stage6heatmap_output_mean_absolute_error: 1711755.5000
Epoch 00008: saving model to gs://dl_training_results/checkpoints/ModelWeights-01Sun1219-0027-0008.ckpt
281/437 [==================>...........] - ETA: 3:31 - loss: 3513169370137980.5000 - stage1paf_output_loss: 1.5843 - stage2paf_output_loss: 1785.4202 - stage3paf_output_loss: 796023.0625

KeyboardInterrupt: 

In [ ]:
model_path="gs://dl_training_results/models/Model"+datetime.datetime.now().strftime("%d(%a)%m%y-%H%M")
tf.keras.models.save_model(train_model,model_path)

# !!!!!!!IT IS FUCKING WORKING!!!!!!!

In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data

In [ ]:
train_history.history